In [4]:
# from torch_geometric.datasets import MoleculeNet
# import seaborn as sns
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.neighbors import KDTree
from scipy.spatial import cKDTree
# import torch
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from torch import nn
# import torch_geometric.nn as geom_nn
from sklearn.preprocessing import MinMaxScaler
# from torch.utils.data import DataLoader, TensorDataset
import copy
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import pickle

Matplotlib created a temporary cache directory at /localscratch-ssd/287500/matplotlib-0vm0v92w because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
import itertools

def get_periodic_coordinates(coord, size):
    """
    Generate all coordinates within a cubic domain considering periodic boundary conditions.
    
    Parameters:
        coord (pandas dataframe): A pandas dataframe containing the columns (x, y, z) of a point.
        size (int): The size of the cubic domain along each axis.
    Returns:
        list: A list of tuples containing all coordinates within the cubic domain.
    """
    ### make a copy of coord ###
    coord_copy = coord.copy()
    coord_copy = [coord_copy] * 27
    coord_copy = pd.concat(coord_copy, ignore_index=True)
    
    # Get coordinates ###
    if isinstance(coord, pd.DataFrame):
        coord = coord[["x","y","z"]].values

    # Generate all combinations of displacements (-1, 0, 1) along each axis
    displacements = list(itertools.product([-1, 0, 1], repeat=3))

    # Generate all coordinates by applying periodic boundary conditions
    tp_coordinates = list()
    
    for dx, dy, dz in displacements:
        
        temp = list()
        
        for i in range(len(coord)):
            
            x, y, z = coord[i,0],coord[i,1],coord[i,2]  

            new_x = x + dx*size
            new_y = y + dy*size
            new_z = z + dz*size

            temp.append((new_x,new_y,new_z))
            
        tp_coordinates.append( np.array(temp) )
    
    coord_copy[["x","y","z"]] = np.vstack(tp_coordinates)
    
    return np.vstack(tp_coordinates),coord_copy

In [3]:
def group_time_series_data(time_series_data):
    
    """
    Groups the data based on case_ID and time 
    
    Parameters:
       time_series_data (pandas dataframe) : obtained from Ze's final data directory 
    Returns:
        list: A list of pandas dataframes each with a unique case id and time-stamp
    """
    ### load raw data from ze time series data ###
    pd_list  = list()
    
    for (col1_val, col2_val), group in time_series_data.groupby(['case_ID', 'time']):
    
        pd_list.append(group)
    
    return pd_list

In [4]:
def generate_nearest_neighbor_data(time_series_data):

    """
    Wrapper function (in some sense, can be condensed more)to do the data generation 
    
    Parameters:
       time_series_data (pandas dataframe) : obtained from Ze's final data directory 
    Returns:
        list: A list of pandas dataframes each with a unique case id and time-stamp
    """
    
    pd_list = group_time_series_data(time_series_data)
    
    nearest_neighbor_data = list()
    scalar_data = list()
    
    ### Loop over different groups ###
    
    for i in range(len(pd_list)):

        tp_particles = get_periodic_coordinates(pd_list[i],5)
        tree = cKDTree(tp_particles)

        ### Loop over all particles in a group and getting the nearest neighbors ###
        idx = np.stack([ tree.query(pd_list[i].iloc[j][["x","y","z"]].values,16)[1] for j in range(len(pd_list[i])) ])
        nearest_neighbor_data.append(tp_particles[idx])

        ### Getting the scalar data ###
        scalar_data.append( pd_list[i][["Density_ratio","glb_phi","glb_Re","local_Re","Drag"]] )

    nearest_neighbor_data = np.stack(nearest_neighbor_data)
    nearest_neighbor_data = nearest_neighbor_data.reshape(nearest_neighbor_data.shape[0]*nearest_neighbor_data.shape[1]
                                           ,nearest_neighbor_data.shape[2]*nearest_neighbor_data.shape[3])
    
    scalar_data = np.stack(scalar_data)
    scalar_data = scalar_data.reshape(scalar_data.shape[0]*scalar_data.shape[1],scalar_data.shape[2])    
    
    return np.concatenate( (nearest_neighbor_data,scalar_data) ,axis=1)

# Neigh Level Connections

In [5]:
def array_difference(array1,array2):
    
    ### First one must be the bigger array ###
    set1 = set(map(tuple, array1))
    set2 = set(map(tuple, array2))
    
    # Find the set difference
    set_difference = set1 - set2

    # Convert the set difference back to a NumPy array
    return np.array(list(set_difference))

In [6]:
def reshape_to_2d(array):
    """
    Reshape an n-dimensional NumPy array to a 2D array, flattening all dimensions except the last one.

    Parameters:
        array (numpy.ndarray): Input n-dimensional array.

    Returns:
        numpy.ndarray: Reshaped 2D array.
    """
    # Flatten all dimensions except the last one
    new_shape_first_dim = np.prod(array.shape[:-1])
    
    # Keep the last dimension intact
    new_shape_second_dim = array.shape[-1]

    # Reshape the array
    reshaped_array = array.reshape(new_shape_first_dim, new_shape_second_dim)

    return reshaped_array

In [16]:
def generate_nearest_neighbor_data_neigh_levels(poi,tp_particles,num_levels=3,neigh_per_level=[5,3,2]):

    """
    Wrapper function to do the data generation 
    
    Parameters:
       poi (pandas dataframe) : coordinates of the particle of interest 
       tp_particles : the set of all particles (inlcuding particles obatined from peridic shifting)
       num_levels : num of levels of neighborhood/shells wanted (at least 2)
       neigh_per_level : number of neighbors to search per neighborhood/shell
       
    Returns:
        : numpy array with the nearest neighbors and their corresponding index
        : edge indexs, which is a an a tensor of all how all the neighbors are connected
    """

    ### Defining KDtree ###
    tree = KDTree(tp_particles)
    
    ### Loop over different groups ###
    nearest_neighbor_data = list()
    
    for i in range(num_levels):
        
        ### Define query point/points ###
        if i==0:
            
            idx = tree.query(poi,neigh_per_level[0],return_distance=False,sort_results=True)
            nearest_neighbor_data.append(tp_particles[idx][0][1:])
#             print("nearest neighbors addition i=0",tp_particles[idx][0][1:].shape)
            particles_to_remove = np.concatenate( (poi,np.stack(nearest_neighbor_data)[0]) )
            remaining_particles = array_difference(tp_particles,particles_to_remove)
        
        if i>0:
                                                   
            tree = KDTree(remaining_particles)
            query_points = reshape_to_2d( nearest_neighbor_data[-1] ) 

            ### get nearest neighbors ###
            idx = [ tree.query(query_points[j][None,:],neigh_per_level[i],return_distance=False,sort_results=True) for j in range(len(query_points)) ]
            
            ### flattening idx ###
            
            nearest_neighbor_data.append( np.stack([remaining_particles[idx[i]] for i in range(len(idx))]).squeeze(1) )
#             print("nearest neighbors addition i>0",np.stack([remaining_particles[idx[i]] for i in range(len(idx))]).squeeze(1).shape)
            particles_to_remove = np.concatenate( ( particles_to_remove , reshape_to_2d(nearest_neighbor_data[-1]) ) ) 
            remaining_particles = array_difference(remaining_particles,particles_to_remove)

    ### Correcting the shape of the first element ###
    nearest_neighbor_data[0] = nearest_neighbor_data[0][None,:,:]
        
    return nearest_neighbor_data

In [17]:
def generate_list_of_indices_for_particles(poi,nearest_neighbor_data):

    all_involved_particles = np.concatenate( (poi,
                                          reshape_to_2d(nearest_neighbor_data[0]),
                                          reshape_to_2d(nearest_neighbor_data[1]),
                                          reshape_to_2d(nearest_neighbor_data[2]) ) )

    indexes = np.unique(all_involved_particles,axis=0,return_index=True)[1]
    all_involved_particles = np.stack([all_involved_particles[index] for index in sorted(indexes)])
    
    all_involved_particles = np.concatenate( ( np.arange(len(all_involved_particles))[:,None], all_involved_particles ),axis=1 )
    all_involved_particles_pd = pd.DataFrame(all_involved_particles,columns=["Particle No","x","y","z"])

    return all_involved_particles_pd

In [18]:
def generate_connections_single_particle(index_list,particle_of_origin,connected_particles):

    ### idx of the particle of origin ### 
    idx=np.where((index_list["x"].values==particle_of_origin[0])&(index_list["y"].values==particle_of_origin[1])&(index_list["z"].values==particle_of_origin[2]))[0][0]
    origin_id = index_list.iloc[idx]["Particle No"]
    
    ### idx of the connected particles ### 
    idx = (np.stack( [ (index_list["x"].values==connected_particles[i][0])
         &(index_list["y"].values==connected_particles[i][1])
         &(index_list["z"].values==connected_particles[i][2]) for i in range(len(connected_particles))] ))
    
    idx = [np.where(idx[i])[0][0] for i in range(idx.shape[0])]
    
    connected_id = [ index_list.iloc[idx[i]]["Particle No"]  for i in range(len(idx))]

    return origin_id,connected_id

In [19]:
def generate_connections_multiple_particle(index_list,nearest_neighbor_data,num_levels,neigh_per_level):

    ### Define edge connectors list ###
    edge_index = list()

    ### iterate over consecutive neighborhood levels ###
    for i in range(num_levels-1):

        ### Define temporary previous and next levels (reshape the previous to only dimensions)###
        previous_level = reshape_to_2d( nearest_neighbor_data[i] )
        next_level = nearest_neighbor_data[i+1] 
        
        for j in range(len(previous_level)):
            
            edge_index.append( generate_connections_single_particle(index_list = index_list,
                             particle_of_origin = previous_level[j],
                             connected_particles = next_level[j]) )
    
    edge_index = edge_index_post_proc(edge_index)
    
    ### add connections from POI to the first level neighbors ###
    edge_index = [([0,i]) for i in range(neigh_per_level[0]-1)] + edge_index
    
    return edge_index

In [20]:
def merge_columns_to_pandas_list(pandas_dataframe,variable_list,master_dataframe):

    """ given a list of pandas dataframe with the x,y,z locations and re and phi ,this function will
        merge each pandas dataframe from the list with the master dataframe with all the columns  
    """

    joined = copy.deepcopy(pandas_dataframe)
    add = pd.merge(pandas_dataframe,master_dataframe,how="inner",on=["x","y","z"],sort=False)[variable_list]
        
    return  pd.concat([joined,add], axis=1)

In [21]:
def edge_index_post_proc(edge_index):
    
    return [( [edge_index[i][0] , edge_index[i][1][j] ] ) for i in range(len(edge_index)) for j in range(len(edge_index[i][1]))]

In [26]:
def final_wrapper(pd_list,num_levels=3,neigh_per_level=[5,3,2]):

    nodes_list = list()
    connections_list = list()
    
    ### iterate over case_time subgroups ###
#     for i in range(50):
    for i in range(len(pd_list)):
        
        tp_particles,tp_particles_dataframe = get_periodic_coordinates(pd_list[i],5)

        ### iterate over the particles in each case_time subgroup ###
        for j in range(pd_list[i].shape[0]):
            
            print("Currently on case_time subgroup : ",str(i+1), "and on particle number : ",str(j+1))
            poi = np.array(pd_list[i].iloc[j][["x","y","z"]].values[None,:])

            ### Generate nearest neighbors data ###
            nearest_neighbor_data = generate_nearest_neighbor_data_neigh_levels(poi,tp_particles,num_levels=num_levels
                                                                                ,neigh_per_level=neigh_per_level)
            
            ### Find unique data particles and assign an index to each of them ###
            all_involved_particles_pd = generate_list_of_indices_for_particles(poi,nearest_neighbor_data)
            
            ### Generate connection indexes ###
            edge_index = generate_connections_multiple_particle(all_involved_particles_pd,nearest_neighbor_data
                                                                ,num_levels=num_levels,neigh_per_level=neigh_per_level)

            ### Append local features to all_involved_particles_pd ###
            all_involved_particles_pd_extra = merge_columns_to_pandas_list(all_involved_particles_pd
                                                                          ,variable_list=["local_Re"],
                                                                           master_dataframe=tp_particles_dataframe)
          
            ### append data to lists ###
            nodes_list.append(all_involved_particles_pd_extra)
            connections_list.append(edge_index)
            clear_output(wait=True)

    return nodes_list,connections_list,nearest_neighbor_data

In [27]:
### Read data ###
time_series_data = pd.read_csv("/home/neilashwinraj/gnns/ze_time_series_data_raw/rho100_10percent_Re100.dat")
pd_list = group_time_series_data(time_series_data)
nodes_list,connections_list,nearest_neighbor_data = final_wrapper(pd_list)

Currently on case_time subgroup :  984 and on particle number :  24


End

# Quick Train

In [33]:
with open('nodes_list.pkl', 'rb') as file:
    node_list = pickle.load(file)
    
with open('connections_list.pkl', 'rb') as file:
    connections_list = pickle.load(file)

### Pandas to numpy array ###
# node_list = [np.array(node_list[i].values[:,1:]) for i in range(len(node_list))]

In [34]:
def split_array_randomly(node_list, n):
    # Create the array from 0 to len(node_list) - 1
    array = np.arange(len(node_list))
    
    # Shuffle the array randomly
    array = np.random.shuffle(array)
    
    # Compute the split index
    split_index = int(len(array) * n / 100)
    
    # Split the array
    part1 = array[:split_index]
    part2 = array[split_index:]
    
    return part1, part2

In [35]:
def min_max_scaler(data,train_indices,test_indices):

    ### Extracting Train Dataset ###
    train_dat = [ data[train_indices[i]] for i in range(len(train_indices)) ]

    ### Extracting Test Dataset ###
    test_dat = [ data[test_indices[i]] for i in range(len(test_indices)) ]

    ### Scaling inputs ###
    ### Getting min and max values ###
    max_vals = np.max(np.stack([train_dat[i].x[:,j].max() for i in 
                                range(len(train_dat)) for j in range(train_dat[i].x.shape[1]) ]).reshape(len(train_dat),train_dat[0].x.shape[1]),axis=0)
    
    min_vals = np.min(np.stack([train_dat[i].x[:,j].min() for i in 
                                range(len(train_dat)) for j in range(train_dat[i].x.shape[1]) ]).reshape(len(train_dat),train_dat[0].x.shape[1]),axis=0)
    
    ### Performing the scaling ###
    train_dat_input = [ (train_dat[i].x - min_vals)/(max_vals - min_vals) for i in range(len(train_dat))] 
    test_dat_input = [ (test_dat[i].x - min_vals)/(max_vals - min_vals) for i in range(len(test_dat))]

    ### Scaling outputs ###
    ### Getting min and max values ###
    max_vals = np.max( np.stack( [train_dat[i].y.max() for i in range(len(train_dat))] ),axis=0)
    min_vals = np.min( np.stack( [train_dat[i].y.min() for i in range(len(train_dat))] ),axis=0)

    ### Performing the scaling ###
    train_dat_output = [ (train_dat[i].y - min_vals)/(max_vals - min_vals) for i in range(len(train_dat))] 
    test_dat_output = [ (test_dat[i].y - min_vals)/(max_vals - min_vals) for i in range(len(test_dat))]

    ### Combine x and y edge indices from the train and test dataset to form a pygnn Data variable ###
    train_combined = list()
    test_combined = list()
    
    for i in range(len(train_dat_input)):

        x = train_dat_input[i].clone()
        edge_index = (train_dat[i].edge_index).clone()
        y = (train_dat_output[i]).clone()
        train_combined.append(Data(x=x , edge_index=edge_index , y=y))

    for i in range(len(test_dat_input)):

        x = test_dat_input[i].clone()
        edge_index = (test_dat[i].edge_index).clone()
        y = (test_dat_output[i]).clone()
        test_combined.append(Data(x=x , edge_index=edge_index , y=y))

    return train_combined,test_combined

In [38]:
### Data reshaping ###
# train_indices,test_indices = split_array_randomly(node_list,80)

### Data scaling ###
train_combined,test_combined = min_max_scaler(node_list,np.arange(100),np.arange(200))

IndexError: tuple index out of range